In [3]:
import os
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
def get_comment(file):
    with open(os.path.join(mydir,file+".txt"), 'r', encoding='utf8') as f:
        return f.read().lower()
    return

In [5]:
mydir = 'C:\\Users\\Sandesh\\Desktop\\hate-speech-dataset\\all_files'
df =pd.read_csv("C:/Users/Sandesh/Desktop/hate-speech-dataset/annotations_metadata.csv")
df['text'] = df['file_id'].apply(lambda x:get_comment(x))

In [6]:
mapping = {'noHate': 1, 'hate':2, 'relation':3,'idk/skip':4 }
df['new_label'] = df['label'].apply(lambda s: mapping.get(s))

In [7]:
df.head()

,file_id,user_id,subforum_id,num_contexts,label,text,new_label
0,12834217_1,572066,1346,0,noHate,"as of march 13th , 2014 , the booklet had been...",1
1,12834217_2,572066,1346,0,noHate,in order to help increase the booklets downloa...,1
2,12834217_3,572066,1346,0,noHate,( simply copy and paste the following text int...,1
3,12834217_4,572066,1346,0,hate,click below for a free download of a colorfull...,2
4,12834217_5,572066,1346,0,noHate,click on the `` download ( 7.42 mb ) '' green ...,1


In [8]:
df1 = df.loc[(df["label"]=='hate') | (df["label"]=='noHate')]

In [49]:
df1.head()

,file_id,user_id,subforum_id,num_contexts,label,text,new_label
0,12834217_1,572066,1346,0,noHate,"as of march 13th , 2014 , the booklet had been...",1
1,12834217_2,572066,1346,0,noHate,in order to help increase the booklets downloa...,1
2,12834217_3,572066,1346,0,noHate,( simply copy and paste the following text int...,1
3,12834217_4,572066,1346,0,hate,click below for a free download of a colorfull...,2
4,12834217_5,572066,1346,0,noHate,click on the `` download ( 7.42 mb ) '' green ...,1


In [76]:
features =  df['text']
target = df['new_label']

In [78]:
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

train = TensorDataset(torch.Tensor(np.array(features)), torch.Tensor(target))
train_loader = DataLoader(train, batch_size = 10, shuffle = True)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: double, float, float16, int64, int32, and uint8.

In [43]:
d = df1[["label",'text']]

In [44]:
d.head()

,label,text
0,noHate,"as of march 13th , 2014 , the booklet had been..."
1,noHate,in order to help increase the booklets downloa...
2,noHate,( simply copy and paste the following text int...
3,hate,click below for a free download of a colorfull...
4,noHate,click on the `` download ( 7.42 mb ) '' green ...


In [2]:
import torch
import torchtext
from torchtext.datasets import text_classification

In [61]:
ds = df1[['label','text']]
train_size = int(0.8 * len(ds))
test_size = len(ds) - train_size
print(train_size, test_size)
train_dataset, test_dataset = torch.utils.data.random_split(ds, [train_size, test_size])

8562 2141


In [65]:
print(len(train_dataset), len(test_dataset))

8562 2141


In [67]:
NGRAMS = 2
import os
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
# train_dataset, test_dataset = text_classification.DATASETS['text'](
#     root='./.data', ngrams=NGRAMS, vocab=None)
BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

train_iterator, valid_iterator, test_iterator = ds.BucketIterator.splits(
    (train_dataset, valid_data, test_dataset), 
    batch_size = BATCH_SIZE, 
    device = device)

AttributeError: 'DataFrame' object has no attribute 'BucketIterator'

In [13]:
import torch.nn as nn
import torch.nn.functional as F
class TextSentiment(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_class):
        super().__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

In [20]:
VOCAB = torchtext.vocab.build_vocab_from_iterator(df1.text)
VOCAB_SIZE = len(VOCAB)
EMBED_DIM = 32
NUN_CLASS = len(df.label.unique())
model = TextSentiment(VOCAB_SIZE, EMBED_DIM, NUN_CLASS).to(device)

10703lines [00:00, 42286.21lines/s]


In [19]:
torchtext.vocab.build_vocab_from_iterator(df1.text)

10703lines [00:00, 34107.13lines/s]


In [45]:
def generate_batch(batch):
    label = torch.tensor([entry[0] for entry in batch])
    text = [entry[1] for entry in batch]
    offsets = [0] + [len(entry) for entry in text]
    # torch.Tensor.cumsum returns the cumulative sum
    # of elements in the dimension dim.
    # torch.Tensor([1.0, 2.0, 3.0]).cumsum(dim=0)

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text = torch.cat(text)
    return text, offsets, label

In [46]:
from torch.utils.data import DataLoader

def train_func(sub_train_):
    
    # Train the model
    train_loss = 0
    train_acc = 0
    data = DataLoader(sub_train_, batch_size=BATCH_SIZE, shuffle=True,
                      collate_fn=generate_batch)
    for i, (text, offsets, cls) in enumerate(data):
        optimizer.zero_grad()
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        output = model(text, offsets)
        loss = criterion(output, cls)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        train_acc += (output.argmax(1) == cls).sum().item()

    # Adjust the learning rate
    scheduler.step()

    return train_loss / len(sub_train_), train_acc / len(sub_train_)

def test(data_):
    loss = 0
    acc = 0
    data = DataLoader(data_, batch_size=BATCH_SIZE, collate_fn=generate_batch)
    for text, offsets, cls in data:
        text, offsets, cls = text.to(device), offsets.to(device), cls.to(device)
        with torch.no_grad():
            output = model(text, offsets)
            loss = criterion(output, cls)
            loss += loss.item()
            acc += (output.argmax(1) == cls).sum().item()

    return loss / len(data_), acc / len(data_)

In [48]:
import time
from torch.utils.data.dataset import random_split
N_EPOCHS = 5
min_valid_loss = float('inf')

criterion = torch.nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=4.0)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1, gamma=0.9)

ds = df1[['label','text']]
train_len = int(len(ds) * 0.80)
sub_train_, sub_valid_ = \
    random_split(ds, [train_len, len(ds) - train_len])
for epoch in range(N_EPOCHS):

    start_time = time.time()
    train_loss, train_acc = train_func(sub_train_)
    valid_loss, valid_acc = test(sub_valid_)

    secs = int(time.time() - start_time)
    mins = secs / 60
    secs = secs % 60

    print('Epoch: %d' %(epoch + 1), " | time in %d minutes, %d seconds" %(mins, secs))
    print(f'\tLoss: {train_loss:.4f}(train)\t|\tAcc: {train_acc * 100:.1f}%(train)')
    print(f'\tLoss: {valid_loss:.4f}(valid)\t|\tAcc: {valid_acc * 100:.1f}%(valid)')

ValueError: Buffer has wrong number of dimensions (expected 1, got 0)

In [27]:
ds = df1[['text','new_label']]
train_len = int(len(ds) * 0.95)
sub_train_, sub_valid_ = \
    random_split(ds, [train_len, len(ds) - train_len])

In [31]:
sub_train_

In [ ]:
dl = DataLoader(
('/data/mnist', train=True, download=True,
                             transform=torchvision.transforms.Compose([
                               torchvision.transforms.ToTensor(),                               
                               torchvision.transforms.Normalize(
                                (0.5,), (0.5,))
                             ])), shuffle=False)

train_ds  = torch.utils.data.Subset(dl.dataset, range(0, 50000-1))
valid_ds = torch.utils.data.Subset(dl.dataset, range(50000, 60000-1))


tensor = dl.dataset.data
tensor = tensor.to(dtype=torch.float32)
tr = tensor.reshape(tensor.size(0), -1) 
tr = tr/128 # tr = tr/255
targets = dl.dataset.targets
targets = targets.to(dtype=torch.long)

In [79]:
import torch
import torchtext
from torchtext.datasets import text_classification
NGRAMS = 2
import os
if not os.path.isdir('./.data'):
    os.mkdir('./.data')
train_dataset, test_dataset = text_classification.DATASETS['AG_NEWS'](
    root='./.data', ngrams=NGRAMS, vocab=None)
BATCH_SIZE = 16
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

.data\ag_news_csv.tar.gz: 11.8MB [00:01, 9.89MB/s]
0lines [00:00, ?lines/s]


OverflowError: Python int too large to convert to C long

In [80]:
TEXT = Field(tokenize=tokenizer, lower=True)
LABEL = LabelField(dtype=torch.float)

trainds = TabularDataset(
    path='drive/{}'.format(TRAIN_PATH), format='tsv',
    fields=[
        ('label_start', LABEL),
        ('label_end', None),
        ('title', None),
        ('symbol', None),
        ('text_content', TEXT),
    ])

testds = TabularDataset(
    path='drive/{}'.format(TEST_PATH), format='tsv',
    fields=[
        ('text_content', TEXT),
    ])

TEXT.build_vocab(trainds, testds)

NameError: name 'Field' is not defined